In [ ]:
from flask import Flask, request, jsonify, send_file, make_response
from werkzeug.exceptions import RequestURITooLarge
from werkzeug.wrappers import Response
import youtube_dl
import os
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

@app.route('/slashing', methods=['GET'])
def slashing_api():
    url = request.args.get('url')
    mp3_file_name = get_mp3(url)
    
    # extract vocal from source.mp3 file
    os.system('echo spleeter_starts')
    os.system('python -m spleeter separate -i source.mp3 -p spleeter:2stems -o output')
    
    # delete source.mp3 file
    os.system('del source.mp3')
    
    resp = make_response(
        send_file("./output/source/vocals.wav", mimetype="audio/wav", as_attachment=True, attachment_filename="vocals.wav")
    )
    
    # send vocal.wav (extracted from source.mp3) to check
    return resp

def get_mp3(url):
    video_info = youtube_dl.YoutubeDL().extract_info(
        url, download=False
    )
    file_name = "source.mp3"
    options = {
        'format' : 'bestaudio/best',
        'keepvideo' : False,
        'outtmpl' : file_name,
        'postprocessors' : [{
            'key' : 'FFmpegExtractAudio',
            'preferredcodec' : 'mp3',
            'preferredquality' : '192',
        }]
    }
    
    with youtube_dl.YoutubeDL(options) as download:
        download.download([url])
        
    return file_name

if __name__ == '__main__':
    app.run(host = '0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


[youtube] mxXvV2ww9Xs: Downloading webpage
[youtube] mxXvV2ww9Xs: Downloading webpage
[download] Destination: source.mp3
[download] 100% of 1.29MiB in 00:00                          
[ffmpeg] Correcting container in "source.mp3"
[ffmpeg] Post-process file source.mp3 exists, skipping


182.211.14.72 - - [10/Apr/2021 15:19:42] "GET /slashing?url=https://www.youtube.com/watch?v=mxXvV2ww9Xs HTTP/1.1" 200 -


[youtube] z6tyssn1Hyg: Downloading webpage
[youtube] z6tyssn1Hyg: Downloading webpage
[download] Destination: source.mp3
[download] 100% of 2.92MiB in 00:00                  
[ffmpeg] Correcting container in "source.mp3"
[ffmpeg] Post-process file source.mp3 exists, skipping
